In [85]:
import random
import json
import numpy as np
import pickle
import tensorflow as tf
import nltk

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', ',', '!', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        documents.append((word_list, intent['tag']))
        words.extend(word_list)  # Append words to the 'words' list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    # Create a bag of words with 1s and 0s
    bag = [1 if word in word_patterns else 0 for word in words]

    # Create the output row
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    # Append the bag and output_row as separate lists
    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Split the training data into X and Y
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('Chatbot_model.h5', hist)
print("Done!")



Epoch 1/200


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MAY02\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MAY02\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\MAY02\anaconda3\envs\openvoice\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - accuracy: 0.0126 - loss: 3.4014    
Epoch 2/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - accuracy: 0.0625 - loss: 3.3209   
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.1204 - loss: 3.3077   
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.1895 - loss: 3.2396
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.2253 - loss: 3.1210
Epoch 6/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.1755 - loss: 3.1001
Epoch 7/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - accuracy: 0.2304 - loss: 2.9566
Epoch 8/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - accuracy: 0.2452 - loss: 2.8714
Epoch 9/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.2069 - loss: 2.7607
Epoch 10/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - accuracy: 0.3319 - loss: 2.5300
Epoch 11/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - accuracy: 0.3374 - loss: 2.4350
Epoch 12/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 

Done!


In [ ]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow import keras 
from keras.models import load_model
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

model = load_model('Chatbot_model.h5')                                            

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] =1 
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r]for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1],reverse= True)
    return_list = []
    for r in results:
        return_list.append({'intents':classes[r[0]],'probaility':str(r[1])})

    return return_list

def get_response(intens_list,intents_json):
    tag = intens_list[0]['intents']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
print("Go! Bot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints,intents)
    print(res)

Go! Bot is running!


 hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Hello


 what


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Date and Time


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
I am Ted, a Deep-Learning chatbot


 who create you\


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Personally, I find Karan very inspiring. I might not be very fair though..


 can solve math 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
I am a general purpose chatbot. My capabilities are : 
 1. I can chat with you. Try asking me for jokes or riddles! 
 2. Ask me the date and time 
 3. I can google search for you. Use format google: your query 
 4. I can get the present weather for any city. Use format weather: city name 
 5. I can get you the top 10 trending news in India. Use keywords 'Latest News' 
 6. I can get you the top 10 trending songs globally. Type 'songs' 
 7. I can set a timer for you. Enter 'set a timer: minutes to timer' 
 8. I can get the present Covid stats for any country. Use 'covid 19: world' or 'covid 19: country name' 
 For suggestions to help me improve, send an email to ted.thedlbot.suggestions@gmail.com . Thank you!! 
